In [9]:
pip install langchain openai pandas numpy scikit-learn yfinance langchain_community matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf



In [11]:
import yfinance as yf
import pandas as pd

def get_tickers_interactive():
    print("Enter stock tickers (e.g., AAPL MSFT CVX):")
    user_input = input().strip().upper()
    return user_input.split()

# Get tickers
clean_tickers = get_tickers_interactive()

# Download data
data = yf.download(
    tickers=clean_tickers,
    period="1y",
    interval="1d",
    group_by='ticker',
    auto_adjust=True,
    progress=False
)

#PRICE_DATA IS ACTUALLY CLOSE PRICE DATA
if len(clean_tickers) == 1:
    # Single ticker case
    returns = data[clean_tickers[0]]['Close'].dropna()
    price_data = pd.DataFrame({
        clean_tickers[0]: returns.values
    })
else:
    # Multiple tickers case
    returns_dict = {}
    for ticker in clean_tickers:
        returns = data[ticker]['Close'].dropna()
        returns_dict[ticker] = returns.values

    price_data = pd.DataFrame(returns_dict)

print("price_data = ({")
for i, col in enumerate(price_data.columns):
    print(f"    '{col}': {price_data[col].tolist()},")
print("})")

# Also display the actual DataFrame
print("\nActual DataFrame:")
print(price_data)

Enter stock tickers (e.g., AAPL MSFT CVX):
AAPL MSFT CVX
price_data = ({
    'AAPL': [193.1267852783203, 193.4453125, 194.95823669433594, 193.57470703125, 195.97349548339844, 192.2210235595703, 206.18572998046875, 212.0782012939453, 213.2427520751953, 211.50086975097656, 215.6614227294922, 213.29248046875, 208.70396423339844, 206.524169921875, 207.17111206054688, 208.09678649902344, 212.25733947753906, 213.10340881347656, 209.6395721435547, 215.74105834960938, 219.24465942382812, 220.51870727539062, 225.28639221191406, 226.75953674316406, 227.61550903320312, 231.89549255371094, 226.5106964111328, 229.46685791015625, 233.3088836669922, 233.72694396972656, 227.81459045410156, 223.13644409179688, 223.26583862304688, 222.91749572753906, 223.96258544921875, 217.52268981933594, 216.4776153564453, 216.94541931152344, 217.22412109375, 217.78150939941406, 221.04623413085938, 217.3435516357422, 218.83656311035156, 208.29586791992188, 206.26535034179688, 208.84332275390625, 212.31704711914062, 21

In [37]:
mean_return = price_data.pct_change().mean()
print(mean_return)
covariance_of_return = price_data.pct_change().cov()
print(covariance_of_return)

AAPL    0.000404
MSFT    0.000607
CVX    -0.000168
dtype: float64
          AAPL      MSFT       CVX
AAPL  0.000436  0.000202  0.000139
MSFT  0.000202  0.000260  0.000066
CVX   0.000139  0.000066  0.000248


In [13]:
covariance_of_close=price_data.cov()
print(covariance_of_close)

            AAPL        MSFT        CVX
AAPL  235.210540   65.117397  44.136961
MSFT   65.117397  536.110716 -17.737207
CVX    44.136961  -17.737207  62.196998


In [14]:
# If you have returns data, convert to price data
close_price_data = yf.download(clean_tickers, period="1y")['Close']
print(close_price_data)



[                       0%                       ]

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  3 of 3 completed

Ticker            AAPL         CVX        MSFT
Date                                          
2024-06-03  193.126816  150.619232  410.373688
2024-06-04  193.445328  149.347031  412.904297
2024-06-05  194.958237  148.246979  420.783875
2024-06-06  193.574707  149.155716  421.289978
2024-06-07  195.973480  149.299194  420.625092
...                ...         ...         ...
2025-05-28  200.419998  136.020004  457.359985
2025-05-29  199.949997  137.910004  458.679993
2025-05-30  200.850006  136.699997  460.359985
2025-06-02  201.699997  137.839996  461.970001
2025-06-03  202.490005  139.925003  462.475006

[251 rows x 3 columns]


In [15]:
pip install PyPortfolioOpt

In [16]:
import numpy as np
import pandas as pd
from pypfopt import (
    EfficientFrontier,
    risk_models,
    expected_returns,
    objective_functions
)

# Clean data
price_data_cleaned = price_data.dropna()

# Calculate parameters
mu = expected_returns.mean_historical_return(price_data_cleaned)
cov_matrix = risk_models.semicovariance(price_data_cleaned)  # Using proper variable name

# Initialize efficient frontier with shorting allowed
ef = EfficientFrontier(mu, cov_matrix, weight_bounds=(-1, 1))

# Add diversification objective
ef.add_objective(objective_functions.L2_reg, gamma=0.1)


try:
    weights = ef.max_sharpe()  # Corrected method name
    cleaned_weights = ef.clean_weights()

    # Print performance metrics
    ef.portfolio_performance(verbose=True)  # Corrected parentheses
    print("\nOptimal weights:", cleaned_weights)

except Exception as e:
    print(f"Optimization failed: {str(e)}")

Expected annual return: 18.6%
Annual volatility: 22.1%
Sharpe Ratio: 0.84

Optimal weights: OrderedDict([('AAPL', 0.48155), ('MSFT', 1.0), ('CVX', -0.48155)])


/usr/local/lib/python3.11/dist-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


In [5]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [17]:
#SHARES TO BE ALLOCATED TO EACH TICKER
portfolio_value=float(input("ENTER THE VALUE YOU ARE LOOKING TO INVEST : "))
latest_prices = get_latest_prices(price_data_cleaned)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
allocation, leftover = da.lp_portfolio()
print(allocation)

ENTER THE VALUE YOU ARE LOOKING TO INVEST : 20000
{'AAPL': 32, 'MSFT': 29, 'CVX': -68}


In [18]:
def get_market_caps(tickers):
    """Get market capitalization for each ticker"""

    market_caps = {}

    for ticker in tickers:
        try:
            # Get stock info
            stock = yf.Ticker(ticker)
            info = stock.info

            # Get market cap
            market_cap = info.get('marketCap', None)
            company_name = info.get('longName', 'N/A')

            if market_cap:
                market_cap_billions = market_cap / 1e9
                market_caps[ticker] = market_cap
                print(f"{ticker}: ${market_cap_billions:.2f}B - {company_name}")
            else:
                print(f"{ticker}: Market cap not available")

        except Exception as e:
            print(f"{ticker}: Error - {e}")

    return market_caps

# Main execution
tickers = clean_tickers
market_caps = get_market_caps(tickers)


AAPL: $3024.95B - Apple Inc.
MSFT: $3437.74B - Microsoft Corporation
CVX: $242.27B - Chevron Corporation


In [26]:
import numpy as np
import pandas as pd

def create_dynamic_bl_inputs(tickers):
    """
    Interactive function to create P matrix and Q vector for Black-Litterman model
    Allows user to input any number of views and any number of assets

    Parameters:
    - tickers: list of ticker symbols in the portfolio

    Returns:
    - P: Picking matrix (n_views x n_assets)
    - Q: Views vector (n_views x 1)
    - view_descriptions: list of view descriptions
    """

    print(f"Creating Black-Litterman views for portfolio: {tickers}")
    print(f"Number of assets: {len(tickers)}")
    print("\n" + "="*60)

    # Get number of views from user
    while True:
        try:
            n_views = int(input("\nHow many views do you want to create? "))
            if n_views > 0:
                break
            else:
                print("Please enter a positive number.")
        except ValueError:
            print("Please enter a valid integer.")

    # Initialize matrices
    n_assets = len(tickers)
    P = np.zeros((n_views, n_assets))
    Q = np.zeros(n_views)
    view_descriptions = []

    print(f"\nAsset indices for reference:")
    for i, ticker in enumerate(tickers):
        print(f"{i}: {ticker}")

    # Collect each view
    for view_idx in range(n_views):
        print(f"\n{'='*40}")
        print(f"CREATING VIEW {view_idx + 1} of {n_views}")
        print(f"{'='*40}")

        # Get view type
        while True:
            view_type = input("""
Choose view type:
1. Absolute view (single asset expected return)
2. Relative view (one asset vs another)
Enter choice (1 or 2): """).strip()

            if view_type in ['1', '2']:
                break
            else:
                print("Please enter 1 or 2")

        if view_type == '1':
            # Absolute view
            P_row, q_val, description = create_absolute_view(tickers)

        elif view_type == '2':
            # Relative view
            P_row, q_val, description = create_relative_view(tickers)


        # Store the view
        P[view_idx, :] = P_row # P_row might be a 1xN array from relative view, need to handle this.
        Q[view_idx] = q_val   # q_val might be a 1x1 array from relative view, need to handle this.
        view_descriptions.append(description) # description might be a list from relative view, need to handle this.

        # Show current view
        print(f"\nView {view_idx + 1} created:")
        print(f"Description: {description}")
        print(f"P row: {P_row}")
        print(f"Q value: {q_val:.4f}")

    return P, Q, view_descriptions

def create_absolute_view(tickers):
    """Create an absolute view for a single asset"""
    print("\n--- Absolute View ---")
    print("This creates a view on the absolute expected return of one asset")

    # Select asset
    while True:
        try:
            asset_choice = input(f"\nSelect asset (enter ticker or index 0-{len(tickers)-1}): ").strip().upper()

            if asset_choice in tickers:
                asset_idx = tickers.index(asset_choice)
                break
            elif asset_choice.isdigit() and 0 <= int(asset_choice) < len(tickers):
                asset_idx = int(asset_choice)
                break
            else:
                print(f"Please enter a valid ticker from {tickers} or index 0-{len(tickers)-1}")
        except:
            print("Invalid input. Please try again.")

    # Get expected return
    while True:
        try:
            expected_return = float(input(f"\nExpected return for {tickers[asset_idx]} (as decimal, e.g., 0.12 for 12%): "))
            break
        except ValueError:
            print("Please enter a valid number.")

    # Create P row (all zeros except 1 for selected asset)
    P_row = np.zeros(len(tickers))
    P_row[asset_idx] = 1.0

    description = f"{tickers[asset_idx]} expected return: {expected_return:.2%}"

    return P_row, expected_return, description

def create_relative_view(tickers):
    """Create one or more relative views between assets"""
    print("\n--- Relative View ---")
    print("This creates views where one asset will outperform another")

    # Get number of views
    while True:
        try:
            # Changed variable name to reflect number of *relative* views to create
            n_relative_views = int(input("\nHow many relative views do you want to create? "))
            if n_relative_views > 0:
                break
            print("Please enter a positive number.")
        except ValueError:
            print("Please enter a valid integer.")

    # Changed initialisation to lists to store multiple views
    P_rows = []
    outperformance_values = []
    descriptions = []

    for i in range(n_relative_views): # Iterate based on the number of relative views
        print(f"\nCreating relative view {i+1}/{n_relative_views}")

        # Select long asset
        while True:
            long_choice = input("Select LONG asset (expected to outperform): ").strip().upper()
            if long_choice in tickers:
                long_idx = tickers.index(long_choice)
                break
            if long_choice.isdigit() and 0 <= int(long_choice) < len(tickers):
                long_idx = int(long_choice)
                break
            print(f"Please enter a valid ticker from {tickers}")

        # Select short asset
        while True:
            short_choice = input("Select SHORT asset (expected to underperform): ").strip().upper()
            if short_choice in tickers:
                short_idx = tickers.index(short_choice)
                if short_idx != long_idx:
                    break
                print("Short asset must be different from long asset.")
            elif short_choice.isdigit() and 0 <= int(short_choice) < len(tickers):
                short_idx = int(short_choice)
                if short_idx != long_idx:
                    break
                print("Short asset must be different from long asset.")
            else:
                print(f"Please enter a valid ticker from {tickers}")

        # Get outperformance
        while True:
            try:
                outperformance = float(input(
                    f"Expected outperformance of {tickers[long_idx]} vs {tickers[short_idx]} (as decimal): "))
                break
            except ValueError:
                print("Please enter a valid number.")

        # Create P row for this relative view
        P_row = np.zeros(len(tickers))
        P_row[long_idx] = 1.0/n_relative_views
        P_row[short_idx] = -1.0/n_relative_views

        # Append the created P_row, outperformance, and description for THIS relative view
        P_rows.append(P_row)
        outperformance_values.append(outperformance)
        descriptions.append(
            f"{tickers[long_idx]} vs {tickers[short_idx]} outperformance: {outperformance:.2%}"
        )

    return P_row, outperformance, descriptions



    # Return the combined matrix, vector, and list of descriptions
    # NOTE: The create_dynamic_bl_inputs function expects a single P_row, q_val, and description
    # This requires a change in how create_dynamic_bl_inputs handles the output of create_relative_view
    # The current structure of create_dynamic_bl_inputs assumes create_relative_view returns a single view.
    # We need to decide if create_relative_view should create multiple rows for one call, or if create_dynamic_bl_inputs
    # should handle multiple calls to create_relative_view for a single view type selection.
    # Based on the original structure (returning P_row, q_val, description), it seems the intent
    # was to create one relative view per loop iteration in create_dynamic_bl_inputs.
    # Let's revise create_relative_view to create just ONE relative view per call.

    # REVISED create_relative_view to create only ONE relative view per call
    print("\n--- Relative View ---")
    print("This creates a single view where one asset will outperform another")


def display_bl_inputs(P, Q, view_descriptions, tickers):
    """Display the final P matrix and Q vector in a nice format"""
    print("\n" + "="*80)
    print("BLACK-LITTERMAN INPUTS CREATED")
    print("="*80)

    print(f"\nNumber of views: {len(Q)}")
    print(f"Number of assets: {len(tickers)}")

    # Display P matrix
    print(f"\nPICKING MATRIX (P):")
    print(f"Shape: {P.shape}")
    P_df = pd.DataFrame(P,
                       columns=tickers,
                       index=[f"View_{i+1}" for i in range(len(Q))])
    print(P_df.round(4))


    # Display Q vector
    print(f"\nVIEWS VECTOR (Q):")
    Q_df = pd.DataFrame({
        # 'View': [f"View_{i+1}" for i in range(len(Q))],
        # 'Description': view_descriptions,
        'Expected_Return': Q,
        'Expected_Return_%': Q*100,
    })
    print(Q_df)

    # Summary statistics
    print(f"\nSUMMARY:")
    print(f"Q vector mean: {np.mean(Q):.4f}")
    print(f"Q vector std: {np.std(Q):.4f}")
    print(f"P matrix L1 norms: {np.sum(np.abs(P), axis=1)}")

    return P_df, Q_df

# Main execution function
def main_bl_input_creation(tickers):

    if not isinstance(tickers, list) or len(tickers) == 0:
        raise ValueError("Please provide a non-empty list of tickers")

    print("BLACK-LITTERMAN INPUT GENERATOR")
    print("="*50)

    # Create inputs
    P, Q, descriptions = create_dynamic_bl_inputs(tickers)

    # Display results
    P_df, Q_df = display_bl_inputs(P, Q, descriptions, tickers)

    return P, Q, descriptions, P_df, Q_df


if __name__ == "__main__":

    sample_tickers = clean_tickers
    P, Q, descriptions, P_df, Q_df = main_bl_input_creation(sample_tickers)

BLACK-LITTERMAN INPUT GENERATOR
Creating Black-Litterman views for portfolio: ['AAPL', 'MSFT', 'CVX']
Number of assets: 3


How many views do you want to create? 2

Asset indices for reference:
0: AAPL
1: MSFT
2: CVX

CREATING VIEW 1 of 2

Choose view type:
1. Absolute view (single asset expected return)
2. Relative view (one asset vs another)
Enter choice (1 or 2): 1

--- Absolute View ---
This creates a view on the absolute expected return of one asset

Select asset (enter ticker or index 0-2): 0

Expected return for AAPL (as decimal, e.g., 0.12 for 12%): 0.02

View 1 created:
Description: AAPL expected return: 2.00%
P row: [1. 0. 0.]
Q value: 0.0200

CREATING VIEW 2 of 2

Choose view type:
1. Absolute view (single asset expected return)
2. Relative view (one asset vs another)
Enter choice (1 or 2): 1

--- Absolute View ---
This creates a view on the absolute expected return of one asset

Select asset (enter ticker or index 0-2): 2

Expected return for CVX (as decimal, e.g., 0.12 for

In [27]:
P_df=np.array(P_df)
print(P_df)
Q_df=np.array(Q_df)
print(Q_df)

[[1. 0. 0.]
 [0. 0. 1.]]
[[0.02 2.  ]
 [0.05 5.  ]]


In [58]:
import yfinance as yf
import pandas as pd


market_prices = yf.download("SPY", period="max", auto_adjust=False)["Adj Close"]

# Display the head of the DataFrame to verify
print(market_prices.head())

[*********************100%***********************]  1 of 1 completed

Ticker            SPY
Date                 
1993-01-29  24.452517
1993-02-01  24.626444
1993-02-02  24.678606
1993-02-03  24.939472
1993-02-04  25.043839


In [59]:
import pypfopt
pypfopt.__version__

'1.5.6'

In [74]:
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel, plotting

# Replace 'prices' with the defined variable 'price_data'

delta = black_litterman.market_implied_risk_aversion(market_prices)
delta

print(delta)

3.3153321452765567


In [76]:
from pypfopt.efficient_frontier import EfficientFrontier

In [75]:
from pypfopt import black_litterman, risk_models



prior = black_litterman.market_implied_prior_returns(market_caps, delta, covariance_of_return)

prior=np.array(prior)
print(prior)

[0.00101224 0.00075229 0.00034931]


In [79]:
viewdict = {}
n = len(clean_tickers)
#viewdict is the expected returns which we are expecting

for ticker in clean_tickers:
    viewdict[ticker] = float(input(f"Expected return of {ticker} in decimals: "))

print(viewdict)

bl = BlackLittermanModel(
    cov_matrix=covariance_of_return,
    absolute_views=viewdict,
    pi=prior
)

rets = bl.bl_returns()
ef = EfficientFrontier(rets, cov_matrix)
bl.bl_weights(delta)
weights = bl.clean_weights()

print(weights)

Expected return of AAPL in decimals: 0.4
Expected return of MSFT in decimals: 0.4
Expected return of CVX in decimals: 0.5
{'AAPL': 0.4, 'MSFT': 0.4, 'CVX': 0.5}
OrderedDict([('AAPL', 0.11147), ('MSFT', 0.35525), ('CVX', 0.53328)])
